## 178 org_month_lagあたりのamount, transaction

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [21]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
sys.exit()
# df_hist = utils.read_df_pkl('../input/histori*0*')
df_hist = utils.read_df_pkl('../input/hist_clean*0*')

train_test = utils.reduce_mem_usage(train_test)
df_hist  = utils.reduce_mem_usage(df_hist )

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = utils.reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)

print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)


100%|██████████| 3/3 [00:00<00:00, 147.59it/s]

SystemExit: 

In [20]:
# 期間を絞る
base_month = 0
last_month = 2
col_month = 'org_month_lag'
df = df_hist[df_hist[col_month]<=last_month]
df = df[df[col_month]>=base_month]
df = df[[key, 'org_month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments']]

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
#     del auth1, auth0
#     gc.collect()

### 月を横持ちにして、各月の特徴量にする

In [22]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


feat_cols = ['category_1', 'purchase_amount', 'installments', 'diff_days', 'record']
tmp_list = []

for df, fname in zip([auth1, auth0], ['auth1', 'auth0']):

    df['record'] = 1
    
    aggs = {}
    aggs['category_1'] = ['sum']
    aggs['purchase_amount'] = ['mean', 'sum']
    aggs['installments'] = ['mean']
    aggs['record'] = ['sum']
    
    df['org_month_lag'] =  df['org_month_lag'].map(lambda x:"org_month_lag_" + str(x))
    tmp = df.groupby([key, 'org_month_lag']).agg(aggs)
    new_columns = get_new_columns(fname, aggs)
    tmp.columns = new_columns
    
    use_cols = tmp.columns
    for col in use_cols:
        tmp[col] = tmp[col].astype('float64')
        tmp2 = tmp[col].unstack()
        for month in tmp2.columns:
            tmp2.rename(columns={month:f"{month}_{col}"}, inplace=True)
        tmp_list.append(tmp2)
    
df_yoko = pd.concat(tmp_list, axis=1)
train_test = train_test.join(df_yoko)
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,org_month_lag_0_auth1_category_1_sum,org_month_lag_1_auth1_category_1_sum,org_month_lag_2_auth1_category_1_sum,org_month_lag_0_auth1_purchase_amount_mean,org_month_lag_1_auth1_purchase_amount_mean,org_month_lag_2_auth1_purchase_amount_mean,org_month_lag_0_auth1_purchase_amount_sum,org_month_lag_1_auth1_purchase_amount_sum,org_month_lag_2_auth1_purchase_amount_sum,org_month_lag_0_auth1_installments_mean,org_month_lag_1_auth1_installments_mean,org_month_lag_2_auth1_installments_mean,org_month_lag_0_auth1_record_sum,org_month_lag_1_auth1_record_sum,org_month_lag_2_auth1_record_sum,org_month_lag_0_auth0_category_1_sum,org_month_lag_1_auth0_category_1_sum,org_month_lag_2_auth0_category_1_sum,org_month_lag_0_auth0_purchase_amount_mean,org_month_lag_1_auth0_purchase_amount_mean,org_month_lag_2_auth0_purchase_amount_mean,org_month_lag_0_auth0_purchase_amount_sum,org_month_lag_1_auth0_purchase_amount_sum,org_month_lag_2_auth0_purchase_amount_sum,org_month_lag_0_auth0_installments_mean,org_month_lag_1_auth0_installments_mean,org_month_lag_2_auth0_installments_mean,org_month_lag_0_auth0_record_sum,org_month_lag_1_auth0_record_sum,org_month_lag_2_auth0_record_sum
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,0.0,0.0,0.0,0.056488,0.072021,0.043671,0.169434,3.457031,1.747070,0.000000,0.000000,0.000000,3.0,48.0,40.0,NaN,0.0,0.0,NaN,0.314941,0.033722,NaN,0.314941,0.134888,NaN,0.0,0.25,NaN,1.0,4.0
C_ID_3d0044924f,4,1,0,2017-01,0.392913,8.0,10.0,1.0,0.233643,0.179932,0.254150,4.906250,3.958984,3.050781,1.571289,2.136719,2.333984,21.0,22.0,12.0,NaN,NaN,0.0,NaN,NaN,1.051758,NaN,NaN,1.051758,NaN,NaN,6.00,NaN,NaN,1.0
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_186d6a6901,4,3,0,2017-09,0.142495,0.0,2.0,3.0,0.033295,0.060791,0.450928,0.366211,1.884766,3.156250,1.000000,1.128906,1.428711,11.0,31.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,2.0,7.0,1.0,0.232178,0.104797,0.114258,4.875000,3.248047,5.027344,1.190430,1.032227,1.000000,21.0,31.0,44.0,NaN,NaN,0.0,NaN,NaN,0.234009,NaN,NaN,0.468018,NaN,NaN,1.00,NaN,NaN,2.0


### make ratio feature

In [23]:
fname = '178_l02'
ignore_features = ['first_active_month', 'card_id', target]

for col in train_test.columns:
    if col in ignore_features:
        continue
    if col.count('feature_'):continue
    if not(col.count('month_lag')):continue
    length = len(train_test[col].dropna())
    if length/len(train_test)<0.1:
        print(col)
        continue
        
    train_test[col].fillna(-1, inplace=True)
    feature = train_test[col].values.astype('float32')
    
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)